In [1]:
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np
import pandas as pd

df = pd.read_csv('dataset1GaussianBlurMin4.csv')

# Define the standard hand connections (MediaPipe-style)
hand_connections = [
    (0, 1), (1, 2), (2, 3), (3, 4),     # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),     # Index finger
    (0, 9), (9, 10), (10, 11), (11, 12), # Middle finger
    (0, 13), (13, 14), (14, 15), (15, 16),# Ring finger
    (0, 17), (17, 18), (18, 19), (19, 20) # Pinky finger
    # Optionally, you can add palm connections, e.g., (5,9), (9,13), (13,17)
]

def plot_sample(idx):
    # Get the row corresponding to the selected index
    row = df.iloc[idx]
    
    # Extract the 21 landmarks as (x, y, z) coordinates.
    landmarks = np.array([[row[f'x{i}'], row[f'y{i}'], row[f'z{i}']] for i in range(21)])
    
    # Create a Plotly figure
    fig = go.Figure()
    
    # Add the landmarks as markers
    fig.add_trace(go.Scatter3d(
        x=landmarks[:, 0],
        y=landmarks[:, 1],
        z=landmarks[:, 2],
        mode='markers',
        marker=dict(size=5, color='blue'),
        name='Landmarks'
    ))
    
    # Build lists for line segments that connect the landmarks
    line_x, line_y, line_z = [], [], []
    for start, end in hand_connections:
        # Append the start and end coordinates, then None to break the line
        line_x += [landmarks[start, 0], landmarks[end, 0], None]
        line_y += [landmarks[start, 1], landmarks[end, 1], None]
        line_z += [landmarks[start, 2], landmarks[end, 2], None]
    
    # Add the connections as lines
    fig.add_trace(go.Scatter3d(
        x=line_x,
        y=line_y,
        z=line_z,
        mode='lines',
        line=dict(color='black', width=2),
        name='Hand Skeleton'
    ))
    
    fig.update_layout(
        title=f"Index: {idx} - Label: {row['label']}",
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        margin=dict(l=0, r=0, b=0, t=30)
    )
    
    fig.show()

# Create an interactive slider for selecting a sample index.
interact(plot_sample, idx=widgets.IntSlider(min=0, max=len(df)-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='idx', max=156269), Output()), _dom_classes=('widget-inte…

<function __main__.plot_sample(idx)>

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

set1Path = 'set1Normalized.csv'
set2Path = 'set2Augmented.csv'
set3Path = 'set3Engineered.csv'

set1DF = pd.read_csv(set1Path)
set2DF = pd.read_csv(set2Path)
set3DF = pd.read_csv(set3Path)

X1 = set1DF.drop('label', axis=1)
y1 = set1DF['label']
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, stratify=y1, random_state=123)

X2 = set2DF.drop('label', axis=1)
y2 = set2DF['label']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, stratify=y2, random_state=123)

X3 = set3DF.drop('label', axis=1)
y3 = set3DF['label']
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, stratify=y3, random_state=123)

scaler = StandardScaler()
angle_columns = [col for col in set3DF.columns if col.startswith("angle_")]
X3_train[angle_columns] = scaler.fit_transform(X3_train[angle_columns])
X3_test[angle_columns] = scaler.transform(X3_test[angle_columns])

In [4]:
X3_train

,dist_0_1,dist_1_2,dist_2_3,dist_3_4,dist_0_5,dist_5_6,dist_6_7,dist_7_8,dist_0_9,dist_9_10,...,angle_6_7_8,angle_0_9_10,angle_9_10_11,angle_10_11_12,angle_0_13_14,angle_13_14_15,angle_14_15_16,angle_0_17_18,angle_17_18_19,angle_18_19_20
6947,0.204616,0.215072,0.176093,0.125303,0.520079,0.207746,0.132446,0.115595,0.520272,0.218184,...,-0.760832,-1.261204,-1.005376,-0.927420,-1.537787,-1.660783,-1.322725,-0.809954,-1.394235,-1.440998
10672,0.193582,0.205186,0.179949,0.179479,0.530372,0.247506,0.180722,0.144138,0.522877,0.279390,...,0.639236,-0.621639,-0.443391,-0.058866,-1.168316,-0.890531,-0.135475,-0.683027,-0.909804,-0.226124
57042,0.220020,0.232191,0.143018,0.144591,0.691714,0.351801,0.255246,0.200700,0.665162,0.390516,...,-0.391733,2.501617,-0.701041,-0.876578,2.532915,-1.302028,-1.130491,2.488984,-1.061830,-1.273650
24021,0.189213,0.189422,0.157367,0.129168,0.501281,0.207982,0.141052,0.105490,0.523691,0.229481,...,1.256260,-0.941746,-0.863956,-0.645376,-1.117525,-1.540914,-0.955798,-0.746098,-1.171738,-0.904097
16824,0.165804,0.162913,0.155954,0.133570,0.445818,0.266081,0.175915,0.123904,0.465991,0.321394,...,-0.736099,0.994426,-0.256349,0.225108,0.619116,-0.843426,0.185568,0.024621,-0.657295,0.842186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46796,0.219881,0.233912,0.201990,0.146724,0.495172,0.235022,0.145796,0.128931,0.472545,0.227788,...,-0.323783,-0.656634,-1.018757,-0.511710,1.053563,0.321973,-0.419595,1.512504,0.442921,-0.508185
56654,0.221132,0.210454,0.129131,0.174587,0.638448,0.397848,0.247619,0.180435,0.629954,0.459281,...,-0.575099,2.600702,-0.918160,-0.921458,2.617018,-1.514402,-1.122627,2.594990,-1.175358,-1.104950
51371,0.183760,0.200473,0.187851,0.167037,0.507970,0.240021,0.152174,0.131463,0.453472,0.139625,...,0.059383,0.597751,1.106077,2.699973,0.711195,0.593037,2.689225,0.131804,1.053020,1.452643
112693,0.270238,0.264628,0.230004,0.173469,0.559493,0.192829,0.167553,0.134847,0.534288,0.182393,...,-0.287006,0.340567,1.097708,-0.569070,-0.092858,0.838088,-0.672465,-0.638206,-1.231246,-0.652088


In [ ]:
# Define a hyperparameter grid to search over
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [1.0, 1.5, 2.0]
}

### Tuning for Set 1 (Normalized Raw 63 Features)
print("Tuning for Set 1 (Normalized Raw 63 Features):")
ada1 = AdaBoostClassifier(random_state=123)
grid_search1 = GridSearchCV(ada1, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search1.fit(X1_train, y1_train)

print("Set 1 Best Parameters:", grid_search1.best_params_)
print("Set 1 Best Cross-Validation Accuracy: {:.3f}".format(grid_search1.best_score_))

best_model1 = grid_search1.best_estimator_
y1_pred = best_model1.predict(X1_test)
print("Set 1 Test Accuracy: {:.3f}".format(accuracy_score(y1_test, y1_pred)))
print("Set 1 Classification Report:\n", classification_report(y1_test, y1_pred, zero_division=0))

### Tuning for Set 2 (Augmented / Rotated Features)
print("\nTuning for Set 2 (Augmented / Rotated Features):")
ada2 = AdaBoostClassifier(random_state=123)
grid_search2 = GridSearchCV(ada2, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search2.fit(X2_train, y2_train)

print("Set 2 Best Parameters:", grid_search2.best_params_)
print("Set 2 Best Cross-Validation Accuracy: {:.3f}".format(grid_search2.best_score_))

best_model2 = grid_search2.best_estimator_
y2_pred = best_model2.predict(X2_test)
print("Set 2 Test Accuracy: {:.3f}".format(accuracy_score(y2_test, y2_pred)))
print("Set 2 Classification Report:\n", classification_report(y2_test, y2_pred, zero_division=0))

Tuning for Set 1 (Normalized Raw 63 Features):


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R

Set 1 Best Parameters: {'learning_rate': 2.0, 'n_estimators': 100}
Set 1 Best Cross-Validation Accuracy: 0.267
Set 1 Test Accuracy: 0.398
Set 1 Classification Report:
               precision    recall  f1-score   support

           A       0.13      0.59      0.21       900
           B       0.74      0.03      0.06       900
           C       0.31      0.02      0.03       900
           D       0.30      0.36      0.33       900
           E       0.40      1.00      0.57       900
           F       0.99      0.67      0.80       900
           G       0.53      0.80      0.64       900
           H       0.84      0.29      0.43       900
           I       0.00      0.00      0.00       900
           J       0.00      0.00      0.00       900
           K       0.00      0.00      0.00       900
           L       0.49      0.58      0.53       900
           M       0.95      0.74      0.83       900
           N       0.96      0.12      0.21       900
           O       0.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R

Set 2 Best Parameters: {'learning_rate': 2.0, 'n_estimators': 100}
Set 2 Best Cross-Validation Accuracy: 0.349
Set 2 Test Accuracy: 0.468
Set 2 Classification Report:
               precision    recall  f1-score   support

           A       0.37      0.66      0.47       900
           B       0.79      0.98      0.87       900
           C       0.38      0.55      0.45       900
           D       0.37      0.42      0.40       900
           E       0.44      0.76      0.56       900
           F       0.42      0.14      0.21       900
           G       0.75      0.31      0.44       900
           H       0.70      0.05      0.10       900
           I       0.86      0.19      0.31       900
           J       0.47      0.68      0.55       900
           K       0.86      0.13      0.23       900
           L       0.75      0.84      0.79       900
           M       0.50      1.00      0.66       900
           N       0.00      0.00      0.00       900
           O       0.

In [15]:
temp = AdaBoostClassifier(n_estimators=1000, learning_rate=3.0, random_state=123)
temp.fit(X3_train, y3_train)
y3_pred = temp.predict(X3_test)
print("Test accuracy: {:.3f}".format(accuracy_score(y3_test, y3_pred)))
print("Classification Report:\n", classification_report(y3_test, y3_pred, zero_division=0))

Test accuracy: 0.930
Classification Report:
               precision    recall  f1-score   support

           A       0.92      0.99      0.95       900
           B       1.00      0.99      0.99       900
           C       0.97      0.96      0.97       900
           D       0.92      0.93      0.93       900
           E       1.00      0.99      1.00       900
           F       1.00      0.80      0.89       900
           G       0.92      1.00      0.96       900
           H       0.91      0.39      0.55       900
           I       0.89      0.98      0.93       900
           J       0.98      0.85      0.91       900
           K       1.00      1.00      1.00       900
           L       1.00      0.94      0.97       900
           M       0.86      0.84      0.85       900
           N       0.85      0.80      0.83       900
           O       1.00      0.98      0.99       900
           P       0.90      0.99      0.94       900
           Q       0.98      0.99   

In [13]:
# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [1.0, 1.5, 2.0, 2.5]
}

# Initialize the AdaBoost classifier
ada = AdaBoostClassifier(random_state=123)

# Set up the GridSearchCV (using 5-fold cross-validation)
grid_search = GridSearchCV(ada, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search on Set 3 training data (X3_train, y3_train)
grid_search.fit(X3_train, y3_train)

# Print best parameters and best cross-validation score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy: {:.3f}".format(grid_search.best_score_))

# Retrieve the best estimator and evaluate on the test set
best_model = grid_search.best_estimator_
y3_pred = best_model.predict(X3_test)
print("Test accuracy: {:.3f}".format(accuracy_score(y3_test, y3_pred)))
print("Classification Report:\n", classification_report(y3_test, y3_pred, zero_division=0))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R

Best parameters: {'learning_rate': 2.0, 'n_estimators': 200}
Best cross-validation accuracy: 0.556
Test accuracy: 0.632
Classification Report:
               precision    recall  f1-score   support

           A       0.86      1.00      0.92       900
           B       1.00      1.00      1.00       900
           C       0.00      0.00      0.00       900
           D       0.89      0.22      0.35       900
           E       1.00      0.97      0.99       900
           F       0.00      0.00      0.00       900
           G       0.98      0.99      0.98       900
           H       0.33      0.39      0.36       900
           I       0.54      0.15      0.23       900
           J       0.51      0.87      0.64       900
           K       0.99      0.61      0.75       900
           L       0.98      0.39      0.56       900
           M       0.25      1.00      0.40       900
           N       0.00      0.00      0.00       900
           O       0.55      0.83      0.66  

In [16]:
import numpy as np

# Sigmoid activation function and its derivative
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))

# Initialize input, weights, and bias
x = np.array([[0.5]])  # Input
y = np.array([[1]])    # True label
W1 = np.array([[0.2]]) # Weight for hidden layer
b1 = np.array([[0.1]]) # Bias for hidden layer
W2 = np.array([[0.4]]) # Weight for output layer
b2 = np.array([[0.2]]) # Bias for output layer
eta = 0.1  # Learning rate

# Forward pass
z1 = np.dot(W1, x) + b1
a1 = sigmoid(z1)
z2 = np.dot(W2, a1) + b2
y_pred = sigmoid(z2)

# Compute loss
loss = - (y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

# Backpropagation
delta2 = (y_pred - y)  # Output layer error
dW2 = np.dot(delta2, a1.T)  # Gradient for W2
db2 = delta2  # Gradient for b2

delta1 = np.dot(W2.T, delta2) * sigmoid_derivative(z1)  # Hidden layer error
dW1 = np.dot(delta1, x.T)  # Gradient for W1
db1 = delta1  # Gradient for b1

# Update weights
W2 -= eta * dW2
b2 -= eta * db2
W1 -= eta * dW1
b1 -= eta * db1

print("Updated W1:", W1)
print("Updated b1:", b1)
print("Updated W2:", W2)
print("Updated b2:", b2)


Updated W1: [[0.20196297]]
Updated b1: [[0.10392594]]
Updated W2: [[0.42180271]]
Updated b2: [[0.23965326]]


In [18]:
print("z1:", z1)
print("a1:", a1)
print("z2:", z2)
print("y_pred:", y_pred)
print("Loss:", loss)

z1: [[0.2]]
a1: [[0.549834]]
z2: [[0.4199336]]
y_pred: [[0.60346736]]
Loss: [[0.50506332]]
